# IFC to X3D, Stair example

In [1]:
import os

import ifcopenshell, ifcopenshell.geom, ifcopenshell.util.element
from IPython.display import HTML
from OCC.Extend.DataExchange.X3D import X3DSceneExporter
from OCC.Display.WebGl.jupyter_renderer import create_download_link, progress_indicator
from OCC.Extend.DataExchange.x3d_standard import x3d as XX3D

x3d.py package loaded, have fun with X3D Graphics!


In [2]:
def attributes_to_X3DMetadataSet(attributes_name, the_instance):
    """ export all attributes that start with an uppercase letter,
    convert to a set of metadatastrings
    """
    attributes_metadataset_node = XX3D.MetadataSet(name=attributes_name)
    attributes = dir(the_instance)

    for attr in attributes:
        if attr[0].isupper():
            attr_value = str(getattr(the_instance, attr))
            attr_value = attr_value.replace("'", '"')
            new_metadata = XX3D.MetadataString(name=attr, value=[attr_value])
            attributes_metadataset_node.value.append(new_metadata)

    return attributes_metadataset_node


def dict_to_X3DMetadataSet(the_dict, the_dict_name):
    metadataset_node = XX3D.MetadataSet(name=the_dict_name)
    for key in the_dict:
        if isinstance(the_dict[key], dict):
            new_metadata = dict_to_X3DMetadataSet(the_dict[key], key)
            #metadataset_node.value.append(dict_to_X3DMetadataSet(the_dict[key], key))
        else:
            val = str(the_dict[key])
            val = val.replace("'", '"')
            new_metadata = XX3D.MetadataString(name=key, value=[val])
        metadataset_node.value.append(new_metadata)

    return metadataset_node

In [3]:
ifc_filename = './NBS_PremierLoftLadders_AlumLoftLddrs_Isotec.ifc'

In [4]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)  # tells ifcopenshell to use pythonocc
settings.set(settings.DISABLE_TRIANGULATION, True)  # useless, since it is performed by the Tesselator
settings.set(settings.NO_NORMALS, True)

In [5]:
ifc_file = ifcopenshell.open(ifc_filename)

In [6]:
# create X3D scene
scene_exp = X3DSceneExporter()

In [7]:
# add metadata for project
project = ifc_file.by_type("IfcProject")
project_metadata = attributes_to_X3DMetadataSet('project_attributes', project[0])
scene_exp._x3dscene.children.append(project_metadata)

# add metadata for sites
sites = ifc_file.by_type("IfcSite")
for site in sites:
    site_metadata = attributes_to_X3DMetadataSet('site_attributes', site)
    scene_exp._x3dscene.children.append(site_metadata)

# add metadata for buildings
buildings = ifc_file.by_type("IfcBuilding")
for building in buildings:
    building_metadata = attributes_to_X3DMetadataSet('building_attributes', building)
    scene_exp._x3dscene.children.append(building_metadata)

# add metadata for storeys
storeys = ifc_file.by_type("IfcBuildingStorey")
for story in storeys:
    story_metadata = attributes_to_X3DMetadataSet('storey_attributes', story)
    scene_exp._x3dscene.children.append(story_metadata)

In [8]:
# traverse elements, create geometry is Representation is not None
elements = ifc_file.by_type("IfcElement")
idx_elt = 0
nb_elements = len(elements)
nb_failed = 0

for element in elements:
    # get element properties, psets and create a X3DMetaDataString node
    elem_psets = ifcopenshell.util.element.get_psets(element)
    psets_metadata = dict_to_X3DMetadataSet(elem_psets, "psets")
    attributes_metadata = attributes_to_X3DMetadataSet("element_attributes", element)

    if element.Representation is not None:  # some IfcProducts don't have any 3d representation
        try:
            elmnt_shape = ifcopenshell.geom.create_shape(settings, inst=element)
        except:
            nb_failed += 1
            continue
        
        if elmnt_shape.styles:
            r,g,b,alpha = elmnt_shape.styles[0] # the shape color
            color = (abs(r), abs(g), abs(b))
        else:  #default color, mid gray
            color = (0.5, 0.5, 0.5)

        out_node = scene_exp.add_shape(elmnt_shape.geometry,
                                       shape_color=color,
                                       emissive=False,
                                       export_edges=False)
        #prepend the metadata to the children list
        # the shape geometry metadata
        out_node.children.insert(0, psets_metadata)
        out_node.children.insert(0, attributes_metadata)
    else:
        print(element.ObjectType, "has no representation")
    idx_elt +=1

    # update progress indicator
    progress_indicator(idx_elt / nb_elements)

Progress: [####################] 100.0%


In [9]:
create_download_link(a_str=scene_exp.to_xml(), filename="ifc_stair.x3d")

HTML(value='<a download="ifc_stair.x3d" href="data:text/x3d;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVR…

In [ ]:
HTML(scene_exp.to_x3dom_html())